<a href="https://colab.research.google.com/github/R329aghav/Raghavjr/blob/main/Keras_Tuner_Hypermeter_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import keras
from tensorflow import keras 
from tensorflow.keras import layers
# from kerastuner.tuners import RandomSearch
from keras.layers import Dense




In [ ]:
!pip install keras-tuner 

     |████████████████████████████████| 97 kB 4.4 MB/s 


In [ ]:
path="/content/drive/MyDrive/DATASETS/Air Quality Index- Delhi.csv"
data=pd.read_csv(path)


In [ ]:
data.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [ ]:
X=data.iloc[:,:-1].values#independent variable
y=data.iloc[:,-1].values#dependent variable

In [ ]:
## Hyper parameter
# 1 how many number of hidden layers we should have
# 2 how many number of neurons we should have
# 3 Learning Rate

In [ ]:
def build_model(self,hp):
  model=keras.Sequential()
  for i in range(hp.Int('number_layers',2,20)):
    model.add(Dense(units=hp.Int('units_' + str(i),min_value=32,max_value=512),steps=32),activation='relu')
    model.add(Dense(1,activation='linear'))
    model.compile(optimizer=keras.optimizers.Adam(hp.choice('learning_rate',[1e-2,1e-3,1e-4])),loss='mean_absolute_error',metrics=['mean_absolute_error'])
    return model

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
!pip install -q -U keras-tuner

In [ ]:
import keras_tuner as kt

In [ ]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [ ]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model


In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [ ]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 31 Complete [00h 00m 39s]
val_accuracy: 0.89083331823349

Best val_accuracy So Far: 0.89083331823349
Total elapsed time: 00h 13m 33s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 160 and the optimal learning rate for the optimizer
is 0.001.



In [ ]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.5165 - accuracy: 0.8178 - val_loss: 0.4215 - val_accuracy: 0.8446
Epoch 2/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3849 - accuracy: 0.8615 - val_loss: 0.3775 - val_accuracy: 0.8658
Epoch 3/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3459 - accuracy: 0.8722 - val_loss: 0.3520 - val_accuracy: 0.8713
Epoch 4/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3186 - accuracy: 0.8831 - val_loss: 0.3614 - val_accuracy: 0.8686
Epoch 5/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2995 - accuracy: 0.8900 - val_loss: 0.3347 - val_accuracy: 0.8790
Epoch 6/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2824 - accuracy: 0.8954 - val_loss: 0.3346 - val_accuracy: 0.8795
Epoch 7/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2699 - accuracy: 0.9007 - val_loss: 0.3249 - val_accuracy: